In [4]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc for doc in docs)

a = {"question": "sdf", "documents": ["bgf", "fgsf", "fsfs"]}
tmp = itemgetter("question")
tmp1 = lambda input: format_docs(input["documents"])

In [5]:
tmp1(a)

'bgf\n\nfgsf\n\nfsfs'

In [2]:
tmp(a)

'sdf'

In [1]:
import os

In [6]:
os.path.join("./retrieval/", "routers.py")

'./retrieval/routers.py'

In [1]:
for k, v in {"a": "b"}.items():
    print(k)
    print(v)

a
b


In [1]:
from src.retrieval.utils import docs_chunking
from langchain.text_splitter import TokenTextSplitter

In [2]:
input_dir = "/home/vinhnq29/Public/advanced_RAG/langchain/db/wiki/docs"
extension = "text"
output_dir = "/home/vinhnq29/Public/advanced_RAG/langchain/db/wiki/chunks"
text_splitter = TokenTextSplitter(chunk_size=256, chunk_overlap=10)

def add_chunk_metadata(doc: dict, chunk_text: str) -> dict:
    chunk_metadata = {}
    for k, v in doc.items():
        chunk_metadata[f"doc_{k}"] = v
    chunk_metadata.pop("doc_text")
    return chunk_metadata

In [3]:
texts, metadatas = docs_chunking(
    input_dir=input_dir,
    extension=extension,
    output_dir=output_dir,
    text_splitter=text_splitter,
    add_chunk_metadata=add_chunk_metadata
)

In [ ]:
text

In [ ]:
# import nltk

# nltk.download('punkt')

In [12]:
import pandas as pd
import glob
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
tqdm.pandas()

In [4]:
data = []

for filepath in glob.glob("./db/wiki/docs/*.parquet"):
    data.append(pd.read_parquet(filepath))
    
data = pd.concat(data, axis=0, ignore_index=True)

In [16]:
data["n_sents"] = data["text"].progress_apply(lambda text: len(sent_tokenize(text)))

100%|██████████| 1287811/1287811 [02:50<00:00, 7570.70it/s] 


In [20]:
data["n_words"] = data["text"].progress_apply(lambda text: len(text.split()))

100%|██████████| 1287811/1287811 [00:11<00:00, 111453.57it/s]


In [33]:
data[data["n_words"]>50]

,id,revid,url,title,text,n_paragraphs,n_sents,n_words
88,1321224,686003,https://vi.wikipedia.org/wiki?curid=1321224,"Yatağan, Muğla","Yatağan là một huyện thuộc tỉnh Muğla, Thổ Nhĩ...",3,5,132
119,1321256,790982,https://vi.wikipedia.org/wiki?curid=1321256,Phó tế,"Phó tế hay thầy sáu trong tiếng Việt cổ, cũng ...",3,4,167
120,1321259,679363,https://vi.wikipedia.org/wiki?curid=1321259,Cừu Bharal,"Cừu Bharal (danh pháp hai phần: ""Pseudois naya...",3,8,151
121,1321262,70336050,https://vi.wikipedia.org/wiki?curid=1321262,Nhạn bụng trắng,Nhạn bụng trắng hay nhạn nhà (danh pháp hai ph...,3,10,311
122,1321283,874327,https://vi.wikipedia.org/wiki?curid=1321283,Johann Heinrich von Thünen,Johann Heinrich von Thünen (24 tháng 6 năm 178...,3,3,144
...,...,...,...,...,...,...,...,...
1287655,1320961,721305,https://vi.wikipedia.org/wiki?curid=1320961,Kocaeli,"Kocaeli là một thành phố tự trị (""büyük şehir""...",3,10,179
1287656,1320966,686003,https://vi.wikipedia.org/wiki?curid=1320966,Mardin,"Mardin (tiếng Aramaic: ܡܶܪܕܺܝܢ, ""Merdin""; Tiến...",9,14,447
1287657,1320968,730776,https://vi.wikipedia.org/wiki?curid=1320968,Muğla,"Muğla là một thành phố tự trị (""büyük şehir"") ...",5,12,212
1287665,1320985,686003,https://vi.wikipedia.org/wiki?curid=1320985,Tunceli,"Tunceli (; , hay ""Mamekiye"")\nlà một thành phố...",10,19,234


In [42]:
links = []
for i, title in tqdm(data[data["n_words"]>50]["title"].items()):
    links.append(f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/vi.wikipedia.org/all-access/all-agents/{title.replace(' ', '_')}/daily/20210101/20240301")

280146it [00:00, 1279898.80it/s]


In [40]:
links[2]

'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/vi.wikipedia.org/all-access/all-agents/Cừu_Bharal/daily/'

In [37]:
data[data["n_words"]>50].to_parquet("./db/wiki/docs/wiki_cleaned.parquet", index=False)

In [43]:
with open('./db/wiki/docs/links.txt', 'w') as f:
    for link in links:
        f.write(f"{link}\n")

In [2]:
[1, 2,3].index(5)

ValueError: 5 is not in list

In [1]:
import json
from tqdm import tqdm

data = {}

with open("./db/wiki/docs/wiki_stats.jsonl") as f:
    for line in tqdm(f):
        try:
            line = json.loads(line)
            line["content"] = json.loads(line["content"])
            article = line["content"]["items"][0]["article"]
            # print(line["content"])
            views = sum([item["views"] for item in line["content"]["items"]])
            if article in data:
                data[article] += views
            else:
                data[article] = views
        except:
            pass
        # data.append(line)
        # break

0it [00:00, ?it/s]

280146it [10:05, 462.58it/s] 


In [5]:
for article, views in data.items():
    print(article)
    print(views)
    break

Yatağan,_Muğla
997


In [8]:
data_sorted = sorted(data.items(), key=lambda x:x[1])

In [18]:
data_sorted[-100000]

('Thành_Gifu', 3268)